#Setup Network Outbound Table

In [0]:
%pip install -r ./requirements.txt -q
dbutils.library.restartPython()

In [0]:
from sentinel_libraries import *

## Environment

In [0]:
dbutils.widgets.text("tenant_id", "")
dbutils.widgets.text("subscription_id", "")
dbutils.widgets.text("sp_client_id", "")
dbutils.widgets.text("sp_client_secret_scope", "")
dbutils.widgets.text("sp_client_secret_key", "")
dbutils.widgets.text("resource_group_name", "")
dbutils.widgets.text("location", "")
dbutils.widgets.text("log_analytics_workspace_name", "")
dbutils.widgets.text("log_analytics_resource_group_name", "")
dbutils.widgets.text("table_name", "AdbSystemAccessOutboundNetwork")

In [0]:
tenant_id = dbutils.widgets.get("tenant_id")
subscription_id = dbutils.widgets.get("subscription_id")
sp_client_id = dbutils.widgets.get("sp_client_id")
sp_client_secret_scope = dbutils.widgets.get("sp_client_secret_scope")
sp_client_secret_key = dbutils.widgets.get("sp_client_secret_key")
resource_group_name = dbutils.widgets.get("resource_group_name")
location = dbutils.widgets.get("location")
log_analytics_workspace_name = dbutils.widgets.get("log_analytics_workspace_name")
log_analytics_resource_group_name = dbutils.widgets.get("log_analytics_resource_group_name")
table_name = dbutils.widgets.get("table_name")

In [0]:
sp_secret = dbutils.secrets.get(sp_client_secret_scope, sp_client_secret_key)
table_retention_in_days = 180

log_analytics_table_name = f"{table_name}_CL"
data_collection_endpoint_name = f"{table_name}-dce"
data_collection_rule_name = f"{table_name}-dcr"
raw_stream_declaration_name = f"Custom-{table_name}RawData"

In [0]:
from azure.identity import ClientSecretCredential, DefaultAzureCredential

credentials = ClientSecretCredential(
    tenant_id=tenant_id,
    client_id=sp_client_id,
    client_secret=sp_secret,
)

### Log Analytics Contributor on the resource group

## Log Analytics Table Creation

In [0]:
log_analytics_table_schema = Schema(
    name=log_analytics_table_name,
    description="Databricks system.access.outbound_network system table",
    columns=[
        Column(name="TimeGenerated", type=ColumnTypeEnum.DATE_TIME),
        Column(name="TimeIngested", type=ColumnTypeEnum.DATE_TIME),
        Column(name="account_id", type=ColumnTypeEnum.STRING),
        Column(name="workspace_id", type=ColumnTypeEnum.STRING),
        Column(name="event_id", type=ColumnTypeEnum.STRING),
        Column(name="destination_type", type=ColumnTypeEnum.STRING),
        Column(name="destination", type=ColumnTypeEnum.STRING),
        Column(name="dns_event", type=ColumnTypeEnum.DYNAMIC),
        Column(name="ip_event", type=ColumnTypeEnum.DYNAMIC),
        Column(name="storage_event", type=ColumnTypeEnum.DYNAMIC),
        Column(name="event_time", type=ColumnTypeEnum.DATE_TIME),
        Column(name="access_type", type=ColumnTypeEnum.STRING),
    ],
)
log_analytics_table = Table(retention_in_days=table_retention_in_days, schema=log_analytics_table_schema)

In [0]:
table = get_table(
    credentials, subscription_id, log_analytics_resource_group_name, log_analytics_workspace_name, log_analytics_table_name
)
if not table:
    create_table(
        log_analytics_table_name,
        log_analytics_table,
        credentials,
        subscription_id,
        log_analytics_resource_group_name,
        log_analytics_workspace_name,
    )
    print(f"Table {log_analytics_table_name} created!")
else:
    print(f"Table creation skipped: {log_analytics_table_name} already exists.")

### Monitoring Contributor on the resource group

## DCE Creation

In [0]:
dce = get_dce(credentials, subscription_id, resource_group_name, data_collection_endpoint_name)
if not dce:
    dce = create_dce(location, credentials, subscription_id, resource_group_name, data_collection_endpoint_name)
    print(f"DCE {data_collection_endpoint_name} created!")
else:
    print(f"DCE creation skipped: {data_collection_endpoint_name} already exists.")

## DCR Creation

In [0]:
dcr = get_dcr(credentials, subscription_id, resource_group_name, data_collection_rule_name)
if not dcr:
    dcr = create_dcr(log_analytics_resource_group_name, log_analytics_workspace_name, log_analytics_table_schema, raw_stream_declaration_name, log_analytics_table_name, location, credentials, subscription_id, resource_group_name, data_collection_rule_name, data_collection_endpoint_name)
    print(f"DCR {data_collection_rule_name} created!")
else:
    print(f"DCR creation skipped: {data_collection_rule_name} already exists.")